Autor: Leonel Criado

Proyecto: Análisis de noticias del Ministerio de Agricultura y Desarrollo Rural (MADR) de Colombia

Creado: 2020-05-19

Meta: Limpieza de la base de datos

In [1]:
# Librerías
import pandas as pd
import numpy as np
import re
import nltk
import string
from nltk import word_tokenize
from nltk.corpus import stopwords

In [2]:
batch1 = pd.read_pickle('df1_news.pkl')
len(batch1)

1000

In [3]:
batch2 = pd.read_pickle('df2_news.pkl')
len(batch2)

1890

In [4]:
news = batch1.append(batch2, ignore_index=True)
print("Número de noticias:",len(news))
news.tail()

Número de noticias: 2890


,dia,mes,año,ubicacion,titulo,url,noticia,seccion
2885,31,07,[2013],N.A.,La “URT” y el IGAC fijan parámetros técnicos p...,https://www.minagricultura.gov.co/noticias/Pag...,BOGOTÁ D.C. - La identificación de los predios...,2
2886,30,07,[2013],N.A.,En más de 600 mil toneladas se incrementó la p...,https://www.minagricultura.gov.co/noticias/Pag...,Un notable avance en la producción de maíz en ...,2
2887,28,07,[2013],N.A.,MINAGRICULTURA Y “DIGNIDAD ARROCERA” EVALÚAN A...,https://www.minagricultura.gov.co/noticias/Pag...,"EL ESPINAL, TOLIMA.- El ministro de Agricultur...",2
2888,25,07,[2013],N.A.,Más de mil familias campesinas del Catatumbo e...,https://www.minagricultura.gov.co/noticias/Pag...,"BOGOTÁ D.C. - Veinte mil millones de pesos, de...",2
2889,25,07,[2013],N.A.,“Se le hace más bien a la sociedad promover el...,https://www.minagricultura.gov.co/noticias/Pag...,"“Vengo a poner la cara, a escucharlos y dentro...",2


In [5]:
type(news)

pandas.core.frame.DataFrame

<div class="alert alert-success">
Limpieza
</div>

### Fecha

In [6]:
news['dia'] = pd.to_numeric(news['dia'], errors = "coerce")
news['mes'] = pd.to_numeric(news['mes'], errors = "coerce")
news['año'] = [''.join(map(str, l)) for l in news['año']]
news['año'] = pd.to_numeric(news['año'], errors = "coerce")

In [7]:
news['dia'].value_counts(dropna=False).sort_index()

1.0      90
2.0      75
3.0      89
4.0     101
5.0     105
6.0     107
7.0      97
8.0      88
9.0      93
10.0    104
11.0     93
12.0     88
13.0     88
14.0     83
15.0    111
16.0     97
17.0     87
18.0     98
19.0     93
20.0    102
21.0     87
22.0     94
23.0     94
24.0     94
25.0     81
26.0    110
27.0    108
28.0    104
29.0     78
30.0     95
31.0     52
NaN       4
Name: dia, dtype: int64

In [8]:
news['mes'].value_counts(dropna=False).sort_index()

1.0     173
2.0     218
3.0     236
4.0     238
5.0     273
6.0     197
7.0     224
8.0     269
9.0     250
10.0    298
11.0    268
12.0    242
NaN       4
Name: mes, dtype: int64

In [9]:
news['año'].value_counts(dropna=False).sort_index()

2013.0    187
2014.0    515
2015.0    492
2016.0    457
2017.0    386
2018.0    472
2019.0    310
2020.0     67
NaN         4
Name: año, dtype: int64

In [10]:
# Formato de fecha
news['fecha'] = pd.to_datetime(news.año * 10000 + news.mes * 100 + news.dia, format='%Y%m%d')

### Missings

In [11]:
news.head()

,dia,mes,año,ubicacion,titulo,url,noticia,seccion,fecha
0,18.0,5.0,2020.0,Bogotá,"Con rigurosos protocolos de bioseguridad, inic...",https://www.minagricultura.gov.co/noticias/Pag...,Con el objeto de blindar el estatus sanitario ...,2,2020-05-18
1,15.0,5.0,2020.0,Bogotá,Con programa de apoyo a la comercialización se...,https://www.minagricultura.gov.co/noticias/Pag...,Se asignaron recursos por $2.700 millones para...,2,2020-05-15
2,15.0,5.0,2020.0,Bogotá,"El sector agropecuario creció 6,8% e impulsó l...",https://www.minagricultura.gov.co/noticias/Pag...,El valor agregado del PIB agropecuario es el s...,2,2020-05-15
3,14.0,5.0,2020.0,Bogotá,MinAgricultura dotará con riego interpredial a...,https://www.minagricultura.gov.co/noticias/Pag...,En el marco del Plan Nacional de Riego y Drena...,2,2020-05-14
4,8.0,5.0,2020.0,Bogotá,MinAgricultura abre convocatoria para el “Camp...,https://www.minagricultura.gov.co/noticias/Pag...,La apuesta es beneficiar a más de 9 mil famili...,2,2020-05-08


In [12]:
news[news["fecha"].isnull()] # Sin fecha

,dia,mes,año,ubicacion,titulo,url,noticia,seccion,fecha
1550,NaN,NaN,NaN,N.A.,N.A,https://www.minagricultura.gov.co/noticias/Pag...,,0,NaT
1551,NaN,NaN,NaN,N.A.,N.A,https://www.minagricultura.gov.co/noticias/Pag...,,0,NaT
2818,NaN,NaN,NaN,N.A.,Convocatoria. Exposicion pública de resultados...,https://www.minagricultura.gov.co/noticias/Pag...,El Programa de Formalización de la Propiedad R...,0,NaT
2859,NaN,NaN,NaN,N.A.,Al menos $15 mil millones necesitaría un campe...,https://www.minagricultura.gov.co/noticias/Pag...,En un debate de control político en la Comisió...,0,NaT


In [13]:
print("Sin titulo:")
print(news["noticia"][news["titulo"] == 'N.A'])
print("Sin noticia:")
print(news["noticia"][news["noticia"] == ""])

Sin titulo:
1550    
1551    
Name: noticia, dtype: object
Sin noticia:
1550    
1551    
Name: noticia, dtype: object


In [14]:
print("Sin acceso:")
print(news["url"][1550])
print(news["url"][1551])
print("Sin fecha:")
print(news["url"][2818])
print(news["url"][2859])

Sin acceso:
https://www.minagricultura.gov.co/noticias/Paginas/familias-rurales-se-atenderan-con-proyecto-ejecutado-por-MinAgricultura.aspx
https://www.minagricultura.gov.co/noticias/Paginas/Avanza-di%C3%A1logo-entre-el-Gobierno-y-organizaciones-campesinas-del-Catatumbo.aspx
Sin fecha:
https://www.minagricultura.gov.co/noticias/Paginas/Convocatoria.%20Exposicion%20p%C3%BAblica%20de%20resultados%20preliminares%20en%20el%20departamento%20del%20Cauca.aspx
https://www.minagricultura.gov.co/noticias/Paginas/Al-menos-$15-mil-millones-necesitar%C3%ADa-un-campesino-para-la-adaptaci%C3%B3n-de-una-%E2%80%9Cuaf%E2%80%9D-en-la-Altillanura-del-Vichada-.aspx


In [15]:
#news.iloc[1551]

In [16]:
news['seccion'].value_counts(dropna=False) 
# Si sección = 2 se capturó la noticia con los patrones fijados de la función remove_tags().

2    2886
0       4
Name: seccion, dtype: int64

In [17]:
print(news["noticia"][news["seccion"] == 0])
print("Noticias capturadas sin seccion:")
print(news["url"][2018]) # Se verificó y la noticia fue capturada completamente.
#print(news["noticia"][2018])
print(news["url"][2059]) # Se verificó y la noticia fue capturada completamente.
#print(news["noticia"][2059])

1550                                                     
1551                                                     
2818    El Programa de Formalización de la Propiedad R...
2859    En un debate de control político en la Comisió...
Name: noticia, dtype: object
Noticias capturadas sin seccion:
https://www.minagricultura.gov.co/noticias/Paginas/agro-de-Santander.aspx
https://www.minagricultura.gov.co/noticias/Paginas/mejorar-en-productividad-y-competitividad.aspx


### Ubicación

In [18]:
news.loc[news['ubicacion'] == 'N.A.','ubicacion'] = np.nan # Reemplazando missings
news['ubicacion'].value_counts(dropna=False)

NaN           1528
Bogotá         772
Rural           40
país            14
Popayán         13
              ... 
entregadas       1
agregado         1
ganaderos        1
1789             1
eficiente        1
Name: ubicacion, Length: 379, dtype: int64

In [19]:
news.ubicacion.unique()

array(['Bogotá', nan, 'm', 'adscritas', 'Vacunación', 'trabajo',
       'firmeza', 'Tunja', 'Rural', 'meta', 'China', 'system',
       'agropecuaria', 'noviembre', 'País', 'Carreño', 'Nacional',
       'aprovechadas', 'Valledupar', 'Tumaco', 'Maíz', 'Dos', 'Medellín',
       'Villanueva', 'Osos', 'semiblanqueado', 'Marta', 'Dibulla',
       'Arroz', '36', 'personas', 'Gobierno', 'Shanghái', 'Quibdó',
       'Agroexpo', 'julio', 'franco', 'títulos', 'Andrés', 'Aires', '5',
       'Montería', 'á', 'colombiano', 'Aracataca', 'Agropecuarios',
       'Cartagena', 'Cali', 'Barranquilla', 'enamorados', 'Palmira',
       'país', 'Guaviare', 'ogotá', 'vez', '2018', 'forma', 'rural',
       'que', 'Soacha', 'Pasto', 'y', 'productores', 'Chinú', 'vida',
       'Malambo', 'Marquetalia', 'Riohacha', 'Octubre', '2017',
       'Villavicencio', '9', 'Filandia', '1', 'septiembre', 'Bucaramanga',
       'Cafeteros', 'Tibú', 'campo', 'Cumbal', 'Buenaventura', 'Ingresos',
       'Quinua', 'Corporación', '

In [20]:
print("Sin ubicación:",news["url"][5])
print("Sin ubicación:",news["url"][29])
print("Sin ubicación:",news["url"][2884])
print("Sin ubicación:",news["url"][2888])

Sin ubicación: https://www.minagricultura.gov.co/noticias/Paginas/Se-posesion%C3%B3-el-nuevo-viceministro-de-Desarrollo-Rural,-Juan-Camilo-Restrepo.aspx
Sin ubicación: https://www.minagricultura.gov.co/noticias/Paginas/%E2%80%9CEl-Gobierno-garantiza-cadena-log%C3%ADstica-de-abastecimiento-de-alimentos,-desde-la-producci%C3%B3n,-transporte-y-distribuci%C3%B3n%E2%80%9D-Min.aspx
Sin ubicación: https://www.minagricultura.gov.co/noticias/Paginas/El-Gobierno-no-estigmatiza-las-zonas-de-reserva-campesina-Minagricultura.aspx
Sin ubicación: https://www.minagricultura.gov.co/noticias/Paginas/M%C3%A1s-de-mil-familias-campesinas-del-Catatumbo-estrenar%C3%A1n-casa,-gracias-al--programa-de-vivienda-rural-del-Gobierno.aspx


### Noticias vacías o duplicadas

In [21]:
news = news[news.noticia != ""] # Vacías
len(news)

2888

In [22]:
news.drop_duplicates(subset = ('noticia'), inplace = True) # Duplicados 1
len(news)

2888

In [23]:
news.drop_duplicates(subset = ('url'), inplace = True) # Duplicados 2
len(news)

2888

In [24]:
news[news.duplicated(subset = ('titulo', 'fecha'))] # Duplicados 3

,dia,mes,año,ubicacion,titulo,url,noticia,seccion,fecha
1429,5.0,8.0,2016.0,NaN,"Con la paz, programas de sustitución de cultiv...",https://www.minagricultura.gov.co/noticias/Pag...,"""la región tiene una inmensa capacidad product...",2,2016-08-05
2748,7.0,11.0,2013.0,NaN,En Quindio Minagricultura busca nuevos cimient...,https://www.minagricultura.gov.co/noticias/Pag...,• Ministro de Agricultura y Desarrollo Rural s...,2,2013-11-07


In [25]:
print(news["titulo"][1429])
print(news["fecha"][news["titulo"] == "Con la paz, programas de sustitución de cultivos ilícitos tendrán mejores resultados"])
print(news["url"][1428])
print(news["url"][1429])
print(news["titulo"][2748])
print(news["fecha"][news["titulo"] == "En Quindio Minagricultura busca nuevos cimientos para la transformacion del agro"])
print(news["url"][2747])
print(news["url"][2748])

Con la paz, programas de sustitución de cultivos ilícitos tendrán mejores resultados
1428   2016-08-05
1429   2016-08-05
Name: fecha, dtype: datetime64[ns]
https://www.minagricultura.gov.co/noticias/Paginas/Con-la-paz,-sustitucion-de-ilicitos-mejores-resultados.aspx
https://www.minagricultura.gov.co/noticias/Paginas/Con-la-paz,-programas-de-sustituci%C3%B3n-de-cultivos-il%C3%ADcitos-tendr%C3%A1n-mejores-resultados.aspx
En Quindio Minagricultura busca nuevos cimientos para la transformacion del agro
2747   2013-11-07
2748   2013-11-07
Name: fecha, dtype: datetime64[ns]
https://www.minagricultura.gov.co/noticias/Paginas/En%20Quindio%20Minagricultura%20busca%20nuevos%20cimientos%20para%20la%20transformacion%20del%20agro.aspx
https://www.minagricultura.gov.co/noticias/Paginas/En-Quindio-Minagricultura-busca-nuevos-cimientos-para-la-transformacion-del-agro.aspx


In [26]:
news.drop_duplicates(subset = ('titulo', 'fecha'), inplace = True, keep = 'last', ignore_index = True)
len(news)

2886

In [27]:
news.tail()

,dia,mes,año,ubicacion,titulo,url,noticia,seccion,fecha
2881,31.0,7.0,2013.0,NaN,La “URT” y el IGAC fijan parámetros técnicos p...,https://www.minagricultura.gov.co/noticias/Pag...,BOGOTÁ D.C. - La identificación de los predios...,2,2013-07-31
2882,30.0,7.0,2013.0,NaN,En más de 600 mil toneladas se incrementó la p...,https://www.minagricultura.gov.co/noticias/Pag...,Un notable avance en la producción de maíz en ...,2,2013-07-30
2883,28.0,7.0,2013.0,NaN,MINAGRICULTURA Y “DIGNIDAD ARROCERA” EVALÚAN A...,https://www.minagricultura.gov.co/noticias/Pag...,"EL ESPINAL, TOLIMA.- El ministro de Agricultur...",2,2013-07-28
2884,25.0,7.0,2013.0,NaN,Más de mil familias campesinas del Catatumbo e...,https://www.minagricultura.gov.co/noticias/Pag...,"BOGOTÁ D.C. - Veinte mil millones de pesos, de...",2,2013-07-25
2885,25.0,7.0,2013.0,NaN,“Se le hace más bien a la sociedad promover el...,https://www.minagricultura.gov.co/noticias/Pag...,"“Vengo a poner la cara, a escucharlos y dentro...",2,2013-07-25


In [28]:
# Ordena por fecha
news.sort_values('fecha', inplace = True)

In [29]:
# Reinicia el índice por fecha
news.reset_index(inplace = True, drop = True)
news.tail()

,dia,mes,año,ubicacion,titulo,url,noticia,seccion,fecha
2881,15.0,5.0,2020.0,Bogotá,"El sector agropecuario creció 6,8% e impulsó l...",https://www.minagricultura.gov.co/noticias/Pag...,El valor agregado del PIB agropecuario es el s...,2,2020-05-15
2882,15.0,5.0,2020.0,Bogotá,Con programa de apoyo a la comercialización se...,https://www.minagricultura.gov.co/noticias/Pag...,Se asignaron recursos por $2.700 millones para...,2,2020-05-15
2883,18.0,5.0,2020.0,Bogotá,"Con rigurosos protocolos de bioseguridad, inic...",https://www.minagricultura.gov.co/noticias/Pag...,Con el objeto de blindar el estatus sanitario ...,2,2020-05-18
2884,NaN,NaN,NaN,NaN,Convocatoria. Exposicion pública de resultados...,https://www.minagricultura.gov.co/noticias/Pag...,El Programa de Formalización de la Propiedad R...,0,NaT
2885,NaN,NaN,NaN,NaN,Al menos $15 mil millones necesitaría un campe...,https://www.minagricultura.gov.co/noticias/Pag...,En un debate de control político en la Comisió...,0,NaT


In [30]:
def clean_texto(año):
    return re.sub('|[|]', '', año)

### Texto de noticias

In [31]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [32]:
puntuacion = string.punctuation + '–¡‘’¿”“•´'
puntuacion

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~–¡‘’¿”“•´'

In [33]:
def no_punt(string):
    strip_punt = string.maketrans('', '', puntuacion)
    return string.translate(strip_punt)

In [34]:
news['sin_punt'] = news['noticia'].apply(no_punt) # Sin puntuación
news['sin_punt'] = news['sin_punt'].apply(str.lower) # Noticia en minúscula
news['tokens'] = news['sin_punt'].apply(word_tokenize) # Noticia tokenizada

In [35]:
stopwords_esp = stopwords.words('spanish')
def stopword(tokens):
    return [w for w in tokens if w not in stopwords_esp]
news['sin_stopwords'] = news['tokens'].apply(stopword) # Sin palabras vacías

In [36]:
# Objeto nltk.Text
news['nltk_Text'] = news['sin_stopwords'].apply(nltk.Text)

In [37]:
news.iloc[0]

dia                                                             25
mes                                                              7
año                                                           2013
ubicacion                                                      NaN
titulo           “Se le hace más bien a la sociedad promover el...
url              https://www.minagricultura.gov.co/noticias/Pag...
noticia          “Vengo a poner la cara, a escucharlos y dentro...
seccion                                                          2
fecha                                          2013-07-25 00:00:00
sin_punt         vengo a poner la cara a escucharlos y dentro d...
tokens           [vengo, a, poner, la, cara, a, escucharlos, y,...
sin_stopwords    [vengo, poner, cara, escucharlos, dentro, posi...
nltk_Text        (vengo, poner, cara, escucharlos, dentro, posi...
Name: 0, dtype: object

In [38]:
news.to_pickle('news.pkl')